<a href="https://colab.research.google.com/github/neonithinar/What-Language-is-this-/blob/main/Europarl_Language_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Language identification using N-grams
We will be using trigrams to identify the language


In [ ]:
# Common Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import time
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from keras.utils import np_utils


Mount the google drive

In [ ]:
! pwd
! mkdir europarl
! cd europarl
! pwd

/content
/content


In [2]:
! wget https://drive.google.com/file/d/1dKnhm9bdUF7KDkvp_IpyPSguXaBBIJFa/view?usp=sharing

--2021-02-04 05:48:26--  https://docs.google.com/u/0/nonceSigner?nonce=6op4hf6pumiqe
Resolving docs.google.com (docs.google.com)... 74.125.142.138, 74.125.142.100, 74.125.142.101, ...
Connecting to docs.google.com (docs.google.com)|74.125.142.138|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://docs.google.com/nonceSigner?nonce=6op4hf6pumiqe [following]
--2021-02-04 05:48:26--  https://docs.google.com/nonceSigner?nonce=6op4hf6pumiqe
Reusing existing connection to docs.google.com:443.
HTTP request sent, awaiting response... 400 Bad Request
2021-02-04 05:48:26 ERROR 400: Bad Request.



In [ ]:
lang = ['bulg', 'dani','germ', 'gree', 'span', 'esto', 'finn', 'fren', 'hung', 'ital', 'lith', 'latv', 'dutc', 'poli', 'port', 'roma', 'slvk', 'slve', 'swed']

In [ ]:
df = pd.DataFrame({'languages': ['bulg', 'dani','germ', 'gree', 'span', 'esto', 'finn', 'fren', 'hung', 'ital', 'lith', 'latv', 'dutc', 'poli', 'port', 'roma', 'slvk', 'slve', 'swed'], 
                   'paths':['/content/drive/MyDrive/Europarl/bg-en.tgz', '/content/drive/MyDrive/Europarl/da-en.tgz', '/content/drive/MyDrive/Europarl/de-en.tgz', '/content/drive/MyDrive/Europarl/el-en.tgz', '/content/drive/MyDrive/Europarl/es-en.tgz', '/content/drive/MyDrive/Europarl/et-en.tgz', '/content/drive/MyDrive/Europarl/fi-en.tgz', '/content/drive/MyDrive/Europarl/fr-en.tgz','/content/drive/MyDrive/Europarl/hu-en.tgz', '/content/drive/MyDrive/Europarl/it-en.tgz', '/content/drive/MyDrive/Europarl/lt-en.tgz', '/content/drive/MyDrive/Europarl/lv-en.tgz','/content/drive/MyDrive/Europarl/nl-en.tgz', '/content/drive/MyDrive/Europarl/pl-en.tgz', '/content/drive/MyDrive/Europarl/pt-en.tgz', '/content/drive/MyDrive/Europarl/ro-en.tgz', '/content/drive/MyDrive/Europarl/sk-en.tgz', '/content/drive/MyDrive/Europarl/sl-en.tgz', '/content/drive/MyDrive/Europarl/sv-en.tgz']})
df
                            

,languages,paths
0,bulg,/content/drive/MyDrive/Europarl/bg-en.tgz
1,dani,/content/drive/MyDrive/Europarl/da-en.tgz
2,germ,/content/drive/MyDrive/Europarl/de-en.tgz
3,gree,/content/drive/MyDrive/Europarl/el-en.tgz
4,span,/content/drive/MyDrive/Europarl/es-en.tgz
5,esto,/content/drive/MyDrive/Europarl/et-en.tgz
6,finn,/content/drive/MyDrive/Europarl/fi-en.tgz
7,fren,/content/drive/MyDrive/Europarl/fr-en.tgz
8,hung,/content/drive/MyDrive/Europarl/hu-en.tgz
9,ital,/content/drive/MyDrive/Europarl/it-en.tgz


In [ ]:
df.loc[3, "paths"]

'/content/drive/MyDrive/Europarl/el-en.tgz'

In [ ]:
import tarfile
import os

def extract_files(tgz_path):
  path  = os.path.join(tgz_path)
  my_tar = tarfile.open(path)
  my_tar.extractall('./europarl') # specify which folder to extract to
  my_tar.close()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Extract the files into the europarl dataset
for i in range(19):
  extract_files(df.loc[i, "paths"])
  

Trying out the text extraction with one of the data files in europarl folder

**UNMOUNT THE GOOGLE DRIVE BEFORE FURTHER PROCESSING**

In [ ]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, mode='rt', encoding='utf-8')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# split a loaded document into sentences
def to_sentences(doc):
	return doc.strip().split('\n')
 
# shortest and longest sentence lengths
def sentence_lengths(sentences):
	lengths = [len(s.split()) for s in sentences]
	return min(lengths), max(lengths)
 
# load English data
filename = '/content/europarl/europarl-v7.fr-en.en'
doc = load_doc(filename)
sentences = to_sentences(doc)
minlen, maxlen = sentence_lengths(sentences)
print('English data: sentences=%d, min=%d, max=%d' % (len(sentences), minlen, maxlen))
print(sentences)

 
# # load French data
# filename = 'europarl-v7.fr-en.fr'
# doc = load_doc(filename)
# sentences = to_sentences(doc)
# minlen, maxlen = sentence_lengths(sentences)
# print('French data: sentences=%d, min=%d, max=%d' % (len(sentences), minlen, maxlen))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

